In [41]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime 
from sklearn.model_selection import train_test_split
import pickle
import xgboost

In [2]:
data = pd.read_csv("Sales.csv")

In [3]:
data.head(10)

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales
0,0,625,3,2013-11-06,641,1,1,0,0,7293
1,1,293,2,2013-07-16,877,1,1,0,1,7060
2,2,39,4,2014-01-23,561,1,1,0,0,4565
3,3,676,4,2013-09-26,1584,1,1,0,0,6380
4,4,709,3,2014-01-22,1477,1,1,0,0,11647
5,5,914,6,2014-10-04,877,1,0,0,0,7967
6,6,1022,3,2013-06-05,789,1,1,0,0,7101
7,8,90,3,2013-02-06,941,1,1,0,0,7516
8,9,860,1,2013-10-21,718,1,1,0,0,6295
9,10,356,4,2014-06-26,317,1,0,0,0,3511


In [4]:
data.dtypes

True_index              int64
Store_ID                int64
Day_of_week             int64
Date                   object
Nb_customers_on_day     int64
Open                    int64
Promotion               int64
State_holiday          object
School_holiday          int64
Sales                   int64
dtype: object

In [4]:
data.shape

(640840, 10)

In [5]:
data = pd.concat([data, pd.get_dummies(data["Day_of_week"])], axis = 1)

In [6]:
data.head(5)

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales,1,2,3,4,5,6,7
0,0,625,3,2013-11-06,641,1,1,0,0,7293,0,0,1,0,0,0,0
1,1,293,2,2013-07-16,877,1,1,0,1,7060,0,1,0,0,0,0,0
2,2,39,4,2014-01-23,561,1,1,0,0,4565,0,0,0,1,0,0,0
3,3,676,4,2013-09-26,1584,1,1,0,0,6380,0,0,0,1,0,0,0
4,4,709,3,2014-01-22,1477,1,1,0,0,11647,0,0,1,0,0,0,0


In [37]:
data.groupby(["State_holiday","Open"]).agg({"Sales":"mean"})

Sales
State_holiday Open              
0             0         0.000000
              1      6957.122912
a             0         0.000000
              1      8452.615385
b             0         0.000000
              1     10449.764706
c             0         0.000000
              1      9763.833333

In [8]:
#Check the initial day and last day:

# data.index = pd.DatetimeIndex(data["Date"])
# data.sort_index(inplace=True)
print(data["Date"].min())
print(data["Date"].max())


2013-01-01 00:00:00
2015-07-31 00:00:00


In [9]:
#Changing date to ordinal number:

data['fixed_Date']=data['Date'].apply(lambda x :x.toordinal())

In [10]:
#Checking for missing values:

data.isnull().sum().sum()

#theres no row with any NaN

0

In [11]:
#Checking values in columns:

    #Promotion:
data["Promotion"].value_counts()

0    396220
1    244620
Name: Promotion, dtype: int64

In [12]:
    #Promotion:
data["State_holiday"].value_counts()

0    621160
a     12842
b      4214
c      2624
Name: State_holiday, dtype: int64

In [13]:
data["Open"].value_counts()

1    532016
0    108824
Name: Open, dtype: int64

In [14]:
data["School_holiday"].value_counts()

0    526468
1    114372
Name: School_holiday, dtype: int64

In [15]:
data = pd.concat([data, pd.get_dummies(data["State_holiday"])], axis = 1)

In [ ]:
data = pd.concat([data, pd.get_dummies(data["State_holiday"])]

In [16]:
data.corr()

,True_index,Store_ID,Day_of_week,Nb_customers_on_day,Open,Promotion,School_holiday,Sales,1,2,3,4,5,6,7,fixed_Date,0,a,b,c
True_index,1.000000,-0.001098,-0.000831,-0.000932,0.000083,-0.001569,-0.000319,-0.000880,0.000226,0.001511,-0.000019,-0.001816,0.000827,-0.000296,-0.000433,-0.001295,-0.000245,0.000607,0.000127,-0.000829
Store_ID,-0.001098,1.000000,0.000329,0.023605,-0.001108,-0.000364,-0.000706,0.004804,0.000565,-0.000310,-0.000217,-0.000452,-0.000622,-0.000231,0.001274,0.000149,-0.000503,0.001005,0.000016,-0.000866
Day_of_week,-0.000831,0.000329,1.000000,-0.385616,-0.527947,-0.392736,-0.206069,-0.461107,-0.611232,-0.409366,-0.204657,-0.000039,0.205213,0.408405,0.611562,0.000785,0.053206,-0.041343,-0.040759,-0.001438
Nb_customers_on_day,-0.000932,0.023605,-0.385616,1.000000,0.617264,0.315858,0.071085,0.894872,0.157166,0.112621,0.077257,0.057364,0.096531,0.020972,-0.523083,0.011718,0.227339,-0.182142,-0.103493,-0.083525
Open,0.000083,-0.001108,-0.527947,0.617264,1.000000,0.294936,0.085213,0.678463,0.130182,0.171875,0.156625,0.101176,0.131319,0.179124,-0.872404,-0.000078,0.379613,-0.303459,-0.174644,-0.138651
Promotion,-0.001569,-0.000364,-0.392736,0.315858,0.294936,1.000000,0.066565,0.451819,0.130844,0.126291,0.126617,0.127710,0.127346,-0.320364,-0.319808,0.017380,0.012813,-0.004241,0.019809,-0.050382
School_holiday,-0.000319,-0.000706,-0.206069,0.071085,0.085213,0.066565,1.000000,0.084484,0.061574,0.076997,0.063323,0.063588,0.073417,-0.169918,-0.169738,0.000357,-0.149864,0.030275,0.158722,0.137570
Sales,-0.000880,0.004804,-0.461107,0.894872,0.678463,0.451819,0.084484,1.000000,0.214951,0.130211,0.082511,0.050796,0.100973,0.007682,-0.588308,0.041123,0.254896,-0.204034,-0.116705,-0.093215
1,0.000226,0.000565,-0.611232,0.157166,0.130182,0.130844,0.061574,0.214951,1.000000,-0.166198,-0.166161,-0.166501,-0.166676,-0.165839,-0.165551,0.000597,-0.042250,0.015762,0.083466,-0.026081
2,0.001511,-0.000310,-0.409366,0.112621,0.171875,0.126291,0.076997,0.130211,-0.166198,1.000000,-0.166921,-0.167262,-0.167439,-0.166598,-0.166309,-0.003427,0.049043,-0.029268,-0.033244,-0.026200


In [ ]:
data.head(5)

In [ ]:
#drop unwanted columns:

data.drop(columns = [ "Day_of_week", "Date","State_holiday"] , axis =1, inplace = True)



In [ ]:
data.columns

In [ ]:
X = data[["School_ID","Nb_customers_on_day','Open', 'Promotion','School_holiday',1,2,3,4,5,6,7,'fixed_Date','0','a','b','c']]

y = data["Sales"]

In [21]:
data["date_year"] = data["Date"].dt.year
data["date_month"] = data["Date"].dt.month
data["date_week"] = data["Date"].dt.week
data["date_day"] = data["Date"].dt.day

<ipython-input-21-747d6eab2f44>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["date_week"] = data["Date"].dt.week


In [22]:
data.head(10)

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales,...,7,fixed_Date,0,a,b,c,date_year,date_month,date_week,date_day
0,0,625,3,2013-11-06,641,1,1,0,0,7293,...,0,735178,1,0,0,0,2013,11,45,6
1,1,293,2,2013-07-16,877,1,1,0,1,7060,...,0,735065,1,0,0,0,2013,7,29,16
2,2,39,4,2014-01-23,561,1,1,0,0,4565,...,0,735256,1,0,0,0,2014,1,4,23
3,3,676,4,2013-09-26,1584,1,1,0,0,6380,...,0,735137,1,0,0,0,2013,9,39,26
4,4,709,3,2014-01-22,1477,1,1,0,0,11647,...,0,735255,1,0,0,0,2014,1,4,22
5,5,914,6,2014-10-04,877,1,0,0,0,7967,...,0,735510,1,0,0,0,2014,10,40,4
6,6,1022,3,2013-06-05,789,1,1,0,0,7101,...,0,735024,1,0,0,0,2013,6,23,5
7,8,90,3,2013-02-06,941,1,1,0,0,7516,...,0,734905,1,0,0,0,2013,2,6,6
8,9,860,1,2013-10-21,718,1,1,0,0,6295,...,0,735162,1,0,0,0,2013,10,43,21
9,10,356,4,2014-06-26,317,1,0,0,0,3511,...,0,735410,1,0,0,0,2014,6,26,26


In [25]:
data["date_year"] = data["Date"].dt.year
data["date_month"] = data["Date"].dt.month
data["date_week"] = data["Date"].dt.week
data["date_day"] = data["Date"].dt.day


data = pd.concat([data, pd.get_dummies(data["date_year"] , prefix = "Year")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["date_month"], prefix = "Month")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["date_week"], prefix = "Week")], axis = 1)
data = pd.concat([data, pd.get_dummies(data["date_day"], prefix = "Day")], axis = 1)

In [32]:
data.head(10)

,True_index,Store_ID,Day_of_week,Date,Nb_customers_on_day,Open,Promotion,State_holiday,School_holiday,Sales,...,22,23,24,25,26,27,28,29,30,31
0,0,625,3,2013-11-06,641,1,1,0,0,7293,...,0,0,0,0,0,0,0,0,0,0
1,1,293,2,2013-07-16,877,1,1,0,1,7060,...,0,0,0,0,0,0,0,0,0,0
2,2,39,4,2014-01-23,561,1,1,0,0,4565,...,0,1,0,0,0,0,0,0,0,0
3,3,676,4,2013-09-26,1584,1,1,0,0,6380,...,0,0,0,0,1,0,0,0,0,0
4,4,709,3,2014-01-22,1477,1,1,0,0,11647,...,1,0,0,0,0,0,0,0,0,0
5,5,914,6,2014-10-04,877,1,0,0,0,7967,...,0,0,0,0,0,0,0,0,0,0
6,6,1022,3,2013-06-05,789,1,1,0,0,7101,...,0,0,0,0,0,0,0,0,0,0
7,8,90,3,2013-02-06,941,1,1,0,0,7516,...,0,0,0,0,0,0,0,0,0,0
8,9,860,1,2013-10-21,718,1,1,0,0,6295,...,0,0,0,0,0,0,0,0,0,0
9,10,356,4,2014-06-26,317,1,0,0,0,3511,...,0,0,0,0,1,0,0,0,0,0


In [42]:
#data = data.drop([data[(data["c"] ==1) & (data["Open"] == 1)]])

#data.drop(["c"], inplace = True)
X = data.drop(columns=["True_index","Day_of_week", "Date", "State_holiday","Sales"], axis=1)

y = data["Sales"]


#First Train:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

xgb_reg = xgboost.XGBRegressor()
    
xgb_reg.fit(X_train, y_train)


#Second train with min max scaler:

normalizer = MinMaxScaler()


X_train_normalized= normalizer.fit(X_train ).transform(X_train)
X_test_normalized = normalizer.fit(X_train).transform(X_test)

X_train_normalized = pd.DataFrame(X_train_normalized , columns = X_train.columns)
X_test_normalized = pd.DataFrame(X_test_normalized , columns = X_test.columns)


xgb_reg = xgboost.XGBRegressor()

xgb_reg.fit(X_train_normalized, y_train)



ValueError: feature_names must be unique

In [43]:

pickle.dump(xgb_reg, open("project_ironkaggle.p", "wb"))
